# 盒鬚圖 - 哪個廣告效果好？ 電商產業廣告效果分析實戰案例

#### 作者：鍾皓軒（臺灣行銷研究創辦人）、徐子皓(臺灣行銷研究特邀作者)
#### 完整文章介紹鏈接：[link](https://medium.com/marketingdatascience/%E7%9B%92%E9%AC%9A%E5%9C%96-%E5%93%AA%E5%80%8B%E5%BB%A3%E5%91%8A%E6%95%88%E6%9E%9C%E5%A5%BD-%E9%9B%BB%E5%95%86%E7%94%A2%E6%A5%AD%E5%BB%A3%E5%91%8A%E6%95%88%E6%9E%9C%E5%88%86%E6%9E%90%E5%AF%A6%E6%88%B0%E6%A1%88%E4%BE%8B-%E9%99%84python-%E7%A8%8B%E5%BC%8F%E7%A2%BC-97d9f411de5c)

#### 本notebook資料可[從此下載](https://drive.google.com/file/d/1qlZcwteIEFfokw-WYEFRSwe5zMDWpLCY/view?usp=sharing)

## 一、基本資料引入

### 1. 引入套件包、原始資料

In [1]:
import pandas as pd
import os
from collections import Counter
import heapq
import plotly.offline as py
import plotly.graph_objects as go

In [2]:
data = pd.read_csv("sales_data2.csv")
data.head(6)

,Unnamed: 0,單價,成本,系列,產品,訂單時間,廣告代號all,去識別化會員編碼
0,0,780.0,370.0,系列5,產品5-3,2016-06-04T07:35:59,廣告_自然流量,L_CBY_02560
1,1,821.0,419.0,系列5,產品5-12,2016-06-04T07:35:59,廣告_自然流量,L_CBY_02560
2,2,494.0,291.0,系列5,產品5-15,2016-06-04T07:35:59,廣告_自然流量,L_CBY_02560
3,3,780.0,370.0,系列215,產品215-3,2016-06-04T07:35:59,廣告_自然流量,L_CBY_02560
4,4,780.0,370.0,系列110,產品110-3,2016-06-04T07:35:59,廣告_自然流量,L_CBY_02560
5,5,164.0,135.0,系列52,產品52-35,2018-09-22T03:41:24,廣告_自然流量,L_CBY_03273


In [5]:
data['廣告代號all'] = data['廣告代號all'].astype(str)

### 2. 移除多餘欄位

In [6]:
data = data.drop(columns = list(data.filter(regex='Unnamed: 0|會員|產品')))
data.head(6)

,單價,成本,系列,訂單時間,廣告代號all
0,780.0,370.0,系列5,2016-06-04T07:35:59,廣告_自然流量
1,821.0,419.0,系列5,2016-06-04T07:35:59,廣告_自然流量
2,494.0,291.0,系列5,2016-06-04T07:35:59,廣告_自然流量
3,780.0,370.0,系列215,2016-06-04T07:35:59,廣告_自然流量
4,780.0,370.0,系列110,2016-06-04T07:35:59,廣告_自然流量
5,164.0,135.0,系列52,2018-09-22T03:41:24,廣告_自然流量


## 二、基礎資料處理

### 1. 只留下「系列4」的資料

In [7]:
data = data[ data ['系列'] == '系列4']
data.head(6)

,單價,成本,系列,訂單時間,廣告代號all
528,1091.0,472.0,系列4,2017-10-04T07:56:35,廣告_B2KP_D
529,1091.0,472.0,系列4,2017-10-04T07:56:35,廣告_B2KP_D
530,1091.0,472.0,系列4,2017-10-04T07:56:35,廣告_B2KP_D
654,1135.0,661.0,系列4,2018-01-05T04:13:39,廣告_critei_critei
670,910.0,416.0,系列4,2016-04-04T05:02:11,廣告_ikip_POD
673,910.0,416.0,系列4,2016-04-18T04:57:25,廣告_critei_critei


### 2. 重新整理廣告代號格式

In [8]:
for i in range(0,len(data['廣告代號all'])):
    a = data['廣告代號all'].iloc[i].split('_')[1]
    data['廣告代號all'].iloc[i] = a

c:\Users\howar\anaconda3\envs\MDS\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
data.head(6)

,單價,成本,系列,訂單時間,廣告代號all
528,1091.0,472.0,系列4,2017-10-04T07:56:35,B2KP
529,1091.0,472.0,系列4,2017-10-04T07:56:35,B2KP
530,1091.0,472.0,系列4,2017-10-04T07:56:35,B2KP
654,1135.0,661.0,系列4,2018-01-05T04:13:39,critei
670,910.0,416.0,系列4,2016-04-04T05:02:11,ikip
673,910.0,416.0,系列4,2016-04-18T04:57:25,critei


## 三、進階資料處理－挑選主力廣告

### 1. 抓出廣告數量

In [10]:
len(data['廣告代號all'].unique())

36

### 2. 廣告使用頻率表

In [11]:
count_list = Counter(data['廣告代號all'])
count_list

Counter({'B2KP': 57,
         'critei': 1285,
         'ikip': 652,
         'KBDG': 145,
         '自然流量': 3780,
         'B2KPOD': 23,
         'KDP': 1793,
         'B2K': 452,
         'qdwit': 896,
         'edmMK': 20,
         'KBP': 31,
         'KDPOD': 937,
         'edmP': 128,
         'GINEP': 154,
         'KDMK': 320,
         'edmMP': 26,
         'GINEM': 13,
         'KDM': 2,
         'B2KMP': 1,
         'KDPM': 140,
         'GINEMK': 24,
         'B2KMK': 10,
         'IGP': 20,
         'edmD': 1,
         'YND': 52,
         'KBMP': 1,
         'edmM': 4,
         'GINEMP': 22,
         'KDMP': 29,
         'qpw': 5,
         'cridgewell': 6,
         'edm': 2,
         'pic': 5,
         'B2KM': 1,
         'B2KDG': 21,
         'IGDG': 1})

### 3. 將「count_list」轉換為DataFrame格式

In [12]:
count_list = pd.DataFrame.from_dict(count_list,orient='index').reset_index() # 轉換為DataFrame形式
count_list = count_list.rename(columns = {'index' : '廣告名稱', 0: '樣本數'}) # 重新命名欄位名稱
count_list

,廣告名稱,樣本數
0,B2KP,57
1,critei,1285
2,ikip,652
3,KBDG,145
4,自然流量,3780
5,B2KPOD,23
6,KDP,1793
7,B2K,452
8,qdwit,896
9,edmMK,20


### 4. 取出前三大廣告頻率

In [13]:
heapq.nlargest(3,list(count_list['樣本數']))

[3780, 1793, 1285]

### 5. 抓出第三名的廣告使用頻率

In [14]:
dead_number = heapq.nlargest(4,list(count_list['樣本數']))[2]
dead_number

1285

### 6. 製作出非熱門廣告清單

In [15]:
dead_list = count_list[count_list['樣本數'] < dead_number]
dead_list

,廣告名稱,樣本數
0,B2KP,57
2,ikip,652
3,KBDG,145
5,B2KPOD,23
7,B2K,452
8,qdwit,896
9,edmMK,20
10,KBP,31
11,KDPOD,937
12,edmP,128


### 7. 將非熱門廣告從資料集中排除

In [16]:
for i in dead_list['廣告名稱']:
    data= data[~data['廣告代號all'].isin([i])]

In [17]:
data.head(6)

,單價,成本,系列,訂單時間,廣告代號all
654,1135.0,661.0,系列4,2018-01-05T04:13:39,critei
673,910.0,416.0,系列4,2016-04-18T04:57:25,critei
775,887.0,676.0,系列4,2016-01-13T09:32:27,自然流量
825,686.0,419.0,系列4,2016-08-01T12:35:28,自然流量
826,870.0,419.0,系列4,2016-08-01T12:35:28,自然流量
827,832.0,367.0,系列4,2016-08-01T12:35:28,自然流量


### 8. 將自然流量的交易資料刪除

In [18]:
data = data[~data['廣告代號all'].isin(['自然流量'])]
data.head(6)

,單價,成本,系列,訂單時間,廣告代號all
654,1135.0,661.0,系列4,2018-01-05T04:13:39,critei
673,910.0,416.0,系列4,2016-04-18T04:57:25,critei
995,469.0,199.0,系列4,2016-09-18T18:49:08,critei
1084,642.0,354.0,系列4,2017-05-29T02:16:19,KDP
1134,780.0,367.0,系列4,2016-04-25T06:30:51,critei
1172,576.0,263.0,系列4,2017-04-23T09:47:12,KDP


## 四、進階資料處理－不同廣告每期帶來的淨利

### 1. 新增「月份」欄位

In [19]:
data['月份'] = data['訂單時間'].str.split('-', expand = True)[1].astype(int)
data.head(6)

,單價,成本,系列,訂單時間,廣告代號all,月份
654,1135.0,661.0,系列4,2018-01-05T04:13:39,critei,1
673,910.0,416.0,系列4,2016-04-18T04:57:25,critei,4
995,469.0,199.0,系列4,2016-09-18T18:49:08,critei,9
1084,642.0,354.0,系列4,2017-05-29T02:16:19,KDP,5
1134,780.0,367.0,系列4,2016-04-25T06:30:51,critei,4
1172,576.0,263.0,系列4,2017-04-23T09:47:12,KDP,4


### 2. 新增「年月」欄位

In [20]:
data['年月'] = (data['訂單時間'].str.split('-', expand = True)[0] + data['訂單時間'].str.split('-', expand = True)[1]).astype(int)
data.head(6)

,單價,成本,系列,訂單時間,廣告代號all,月份,年月
654,1135.0,661.0,系列4,2018-01-05T04:13:39,critei,1,201801
673,910.0,416.0,系列4,2016-04-18T04:57:25,critei,4,201604
995,469.0,199.0,系列4,2016-09-18T18:49:08,critei,9,201609
1084,642.0,354.0,系列4,2017-05-29T02:16:19,KDP,5,201705
1134,780.0,367.0,系列4,2016-04-25T06:30:51,critei,4,201604
1172,576.0,263.0,系列4,2017-04-23T09:47:12,KDP,4,201704


### 3. 新增「淨利」欄位

In [21]:
data['淨利'] = data['單價'] - data['成本']
data.head(6)

,單價,成本,系列,訂單時間,廣告代號all,月份,年月,淨利
654,1135.0,661.0,系列4,2018-01-05T04:13:39,critei,1,201801,474.0
673,910.0,416.0,系列4,2016-04-18T04:57:25,critei,4,201604,494.0
995,469.0,199.0,系列4,2016-09-18T18:49:08,critei,9,201609,270.0
1084,642.0,354.0,系列4,2017-05-29T02:16:19,KDP,5,201705,288.0
1134,780.0,367.0,系列4,2016-04-25T06:30:51,critei,4,201604,413.0
1172,576.0,263.0,系列4,2017-04-23T09:47:12,KDP,4,201704,313.0


### 4. 根據「廣告代號all」、「月份」及「年月」欄位交叉組合出不同的淨利總和

In [22]:
data = data.groupby(['廣告代號all', '月份', '年月'])['淨利'].sum()
data

廣告代號all  月份  年月    
KDP      1   201801    46315.0
             201901     1118.0
         2   201702    12109.0
             201802     2825.0
             201902      858.0
                        ...   
critei   11  201711     2319.0
             201811     2176.0
         12  201612    12216.0
             201712     2940.0
             201812     2256.0
Name: 淨利, Length: 68, dtype: float64

### 5. 將「data」重新組合為DataFrame形式

In [23]:
data = data.to_frame().reset_index()
data.head(6)

,廣告代號all,月份,年月,淨利
0,KDP,1,201801,46315.0
1,KDP,1,201901,1118.0
2,KDP,2,201702,12109.0
3,KDP,2,201802,2825.0
4,KDP,2,201902,858.0
5,KDP,3,201703,66600.0


### 6. 移除多餘欄位，並新增「count」欄位

In [24]:
data = data.drop(['年月'], axis=1)
data['count'] = 1
data.head(6)

,廣告代號all,月份,淨利,count
0,KDP,1,46315.0,1
1,KDP,1,1118.0,1
2,KDP,2,12109.0,1
3,KDP,2,2825.0,1
4,KDP,2,858.0,1
5,KDP,3,66600.0,1


## 五、繪圖

### 1. 繪製盒鬚圖

In [25]:
fig = go.Figure() # 設定空白畫布
colors_box = ['#3366CC', '#DC3912'] # 設定盒子顏色
# 繪製盒鬚圖
for i in range(0, len(data['廣告代號all'].unique())):
    fig.add_trace(go.Box(
    y = data[data['廣告代號all'] ==data['廣告代號all'].unique()[i]]['淨利'],
    x = data[data['廣告代號all'] ==data['廣告代號all'].unique()[i]]['月份'],
    name = str(data[data['廣告代號all'] ==data['廣告代號all'].unique()[i]]['廣告代號all'][0:1].values[0]),
    marker_color= colors_box[i]
    ))


### 2. 繪製平均折線圖

In [26]:
adlist = list(data['廣告代號all'].unique()) # 廣告名稱清單
color_line = ["royalblue" , "firebrick"] # 設定線條顏色
# 畫平均線
for i in range(0,len(adlist)):
    ad = adlist[i]
    colour = color_line[i]
    meanlist = [] # 平均
    axislist = [] # 月分
    
    for ii in range(1,13):
        axislist.append(ii)
        total = data[data['廣告代號all'] == ad][data['月份'] == ii]['淨利'].sum()
        counts = data[data['廣告代號all'] == ad][data['月份'] == ii]['count'].sum()
        aver = total/counts
        meanlist.append(aver)
        
        if ii == 12:   
            fig.add_trace(go.Scatter(
            x= axislist,
            y= meanlist,
            mode="lines+markers",
            textfont=dict(
            family="sans serif",
            size=16,
            color="royalblue"),    
            line=dict(color=colour, width=2),
            ))


c:\Users\howar\anaconda3\envs\MDS\lib\site-packages\ipykernel_launcher.py:12: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

c:\Users\howar\anaconda3\envs\MDS\lib\site-packages\ipykernel_launcher.py:13: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



### 3. 設定佈景主題

In [27]:
# 設定佈景主題(字體、大小、背景等)
fig.update_layout(
    title={
        'text': "<b>BoxPlot－系列四 廣告效益分析</b>",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',},
    yaxis_title='Profit',
    xaxis={
        'title': 'Month',
        'tickmode': 'linear'
        },
    width=1800,
    height=960,
    boxmode='group',
    font=dict(
        family="Courier New, monospace",
        size=20,
        color="lightslategrey"
    )
    )
